In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src/'))

### Data preparation

In [2]:
import cicids2017
from pathlib import Path
import pandas as pd
import numpy as np


RES_PATH = "."
MODELPATH = Path(f"../res/{RES_PATH}/ts2vec.torch")
DATASETPATH_CACHE = Path(f"../dataset/cache")
DATASETPATH = Path("../dataset/CICIDS2017_ntop.pkl")
TARGET_DSET = "DT"


netdata = []
df = cicids2017.load_dataset(DATASETPATH_CACHE)[TARGET_DSET]
days = np.unique(df.index.get_level_values("_time").day)
for d in days:
    daily_df = df[df.index.get_level_values("_time").day == d]
    netdata.append(daily_df)

### Prediction

In [3]:
import AnomalyDetector as ad


net = cicids2017.configureAnchor(None, df, checkpoint=MODELPATH)
netdata = net.pointwise_anomaly(netdata)
netdata = pd.concat(netdata)
Y = (netdata["_isanomaly"]=="none")
Y = Y.replace({True: ad.NORMAL_TRAFFIC, False: ad.ATTACK_TRAFFIC }).values

/Users/swedishhdcamera/Desktop/Projects/tesi_sabella/venv/lib/python3.8/site-packages/skorch/dataset.py:267: FutureWarning: Setting a random_state has no effect since cv is not a float. This will raise an error in a future. You should leave random_state to its default (None), or set cv to a float value.
  warnings.warn(


### Results for different sigma

In [4]:
from sklearn import metrics


df_res = pd.DataFrame()
sigma_values = [-.5, -.25, -.05, .0, .05, .25, .5]


for sigma in sigma_values:
    sigma_res = { "sigma": sigma }
    net.module_.sigma = sigma
    netdata = net.pointwise_anomaly(netdata)
    y_hat = pd.concat(netdata)["_y_hat"]
    y_hat = np.round(y_hat)
    
    metrics_rep = [ metrics.roc_auc_score,
                    metrics.precision_score, metrics.recall_score,
                    metrics.accuracy_score, metrics.f1_score]
    for m in metrics_rep:
        mres = m(Y, y_hat)
        sigma_res[m.__name__] = mres
    
    tn, fp, fn, tp = metrics.confusion_matrix(Y, y_hat, normalize="all").ravel()
    sigma_res.update({ "tn": tn, "fp": fp, "fn": fn, "tp": tp })
    sigma_res = pd.DataFrame(sigma_res, index=[0])
    df_res = pd.concat([df_res, sigma_res], ignore_index=True)

### Latex

In [5]:
import re

best_results = df_res.sort_values("roc_auc_score", ascending=False)
latex_str = best_results.to_latex(index=False, float_format="%.3f")
print(re.sub(' +',' ', latex_str))

\begin{tabular}{rrrrrrrrrr}
\toprule
 sigma & roc\_auc\_score & precision\_score & recall\_score & accuracy\_score & f1\_score & tn & fp & fn & tp \\
\midrule
-0.500 & 0.535 & 0.166 & 0.652 & 0.454 & 0.265 & 0.355 & 0.494 & 0.053 & 0.098 \\
 0.000 & 0.531 & 0.160 & 0.930 & 0.253 & 0.273 & 0.113 & 0.737 & 0.011 & 0.140 \\
-0.050 & 0.531 & 0.160 & 0.905 & 0.270 & 0.272 & 0.133 & 0.716 & 0.014 & 0.137 \\
-0.250 & 0.530 & 0.161 & 0.787 & 0.350 & 0.268 & 0.231 & 0.618 & 0.032 & 0.119 \\
 0.050 & 0.528 & 0.159 & 0.949 & 0.234 & 0.272 & 0.090 & 0.759 & 0.008 & 0.143 \\
 0.250 & 0.511 & 0.154 & 0.988 & 0.177 & 0.266 & 0.028 & 0.821 & 0.002 & 0.149 \\
 0.500 & 0.505 & 0.152 & 1.000 & 0.159 & 0.264 & 0.008 & 0.841 & 0.000 & 0.151 \\
\bottomrule
\end{tabular}



In [6]:
best_results

,sigma,roc_auc_score,precision_score,recall_score,accuracy_score,f1_score,tn,fp,fn,tp
0,-0.50,0.535276,0.166175,0.651907,0.453845,0.264840,0.355469,0.493627,0.052529,0.098376
3,0.00,0.531202,0.160023,0.929837,0.252878,0.273055,0.112562,0.736534,0.010588,0.140317
2,-0.05,0.530925,0.160198,0.905313,0.269531,0.272224,0.132915,0.716180,0.014289,0.136616
1,-0.25,0.529530,0.161178,0.786785,0.349918,0.267547,0.231188,0.617907,0.032175,0.118729
4,0.05,0.527724,0.158783,0.948910,0.233655,0.272044,0.090461,0.758635,0.007710,0.143195
5,0.25,0.510614,0.153708,0.988420,0.177015,0.266043,0.027858,0.821238,0.001748,0.149157
6,0.50,0.504782,0.152140,1.000000,0.159025,0.264100,0.008121,0.840975,0.000000,0.150905
